In [3]:
import tensorflow as tf
import numpy as np
from predictor import Predictor
from models import *
import matplotlib.pyplot as plt
import pandas as pd
import math
from tensorflow.keras.layers import *

In [20]:
sections = 6
n = sections + 1
inputs = Input(shape=(2**sections,1), name="inputs")
# x = inputs
l1 = Lambda(lambda z: z[:, -(2 ** sections) :])
l2 = [Lambda(lambda z: z[0][:, -(2 ** z[1]) :]) for i in range(n)]
a=np.arange(128).reshape((2,64))
a1=l1(a)
a2=[l2[i]((a1,i)) for i in range(n)]
a2


[<tf.Tensor: shape=(2, 1), dtype=int64, numpy=
 array([[ 63],
        [127]])>, <tf.Tensor: shape=(2, 2), dtype=int64, numpy=
 array([[ 62,  63],
        [126, 127]])>, <tf.Tensor: shape=(2, 4), dtype=int64, numpy=
 array([[ 60,  61,  62,  63],
        [124, 125, 126, 127]])>, <tf.Tensor: shape=(2, 8), dtype=int64, numpy=
 array([[ 56,  57,  58,  59,  60,  61,  62,  63],
        [120, 121, 122, 123, 124, 125, 126, 127]])>, <tf.Tensor: shape=(2, 16), dtype=int64, numpy=
 array([[ 48,  49,  50,  51,  52,  53,  54,  55,  56,  57,  58,  59,  60,
          61,  62,  63],
        [112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124,
         125, 126, 127]])>, <tf.Tensor: shape=(2, 32), dtype=int64, numpy=
 array([[ 32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44,
          45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  57,
          58,  59,  60,  61,  62,  63],
        [ 96,  97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108,
         109, 11

In [ ]:
x = [Conv1D(16, 2 ** i, padding="valid", name=f"c1-{i}")(x[i]) for i in range(n)]
x = [Lambda(lambda z: z[:, -(2 ** i) :, :])(x) for i in range(n)]
x = [Flatten()(x[i]) for i in range(n)]
x = Concatenate(axis=1)(x)
x = Flatten()(x)
x = Dense(units, "sigmoid")(x)
x = Dense(64, "tanh")(x)
x = Dense(8, "tanh")(x)
x = Dense(input_shape[-1], kernel_regularizer=l2)(x)
outputs = x
model = keras.Model(inputs, outputs, name="trendencoder")
MAE = keras.metrics.MeanAbsoluteError()
RMSE = keras.metrics.RootMeanSquaredError()
model.compile(
    loss=keras.losses.MeanSquaredError(),
    optimizer=keras.optimizers.Adam(learning_rate=1e-2),
    metrics=[MAE],
)
print(model.summary())
